In [1]:
from classes.pipeline.annotator_pipeline import AnnotatorPipeline
annotator_pipeline = AnnotatorPipeline(lexicon_words = [])

word = "many"

print(annotator_pipeline.get_annotated_words(word))

Word(word='many',
definitions=[DefinitionExampleEntry(
  meaning='A multitude; a great aggregate; a mass of people; the generality; the common herd.',
  examples=[
  Democracy must balance the rights of the few against the will of the many.
  ]
), DefinitionExampleEntry(
  meaning='A considerable number.',
  examples=[
  None
  ]
), DefinitionExampleEntry(
  meaning='A large number of; numerous.',
  examples=[
  None
  ]
)], part_of_speech=['ADJECTIVE', 'NOUN', 'PRONOUN'],
sentiment_score=0.046, sentiment_magnitude=0.284, sentiment_type=<SentimentType.NEUTRAL: 'Neutral'>, language=<Language.EN: 'en'>,



In [3]:
from neo4j import GraphDatabase

class Neo4jWordStore:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def store_word_data(self, word_data):
        """
        Store words, their definitions, and part_of_speech in Neo4j.
        """
        with self.driver.session() as session:
            for word_entry in word_data:
                session.write_transaction(self._create_word_node, word_entry)

    @staticmethod
    def _create_word_node(tx, word_entry):
        word = word_entry["word"]
        language = word_entry["language"]
        sentiment_score = word_entry["sentiment_score"]
        sentiment_magnitude = word_entry["sentiment_magnitude"]
        sentiment_type = word_entry["sentiment_type"]
        part_of_speech = word_entry["part_of_speech"]
        definitions = word_entry["definitions"]

        # Create Word Node
        tx.run(
            """
            MERGE (w:Word {word: $word})
            SET w.language = $language,
                w.sentiment_score = $sentiment_score,
                w.sentiment_magnitude = $sentiment_magnitude,
                w.sentiment_type = $sentiment_type
            """,
            word=word,
            language=language,
            sentiment_score=sentiment_score,
            sentiment_magnitude=sentiment_magnitude,
            sentiment_type=sentiment_type
        )

        # Create Part of Speech Nodes and Relationships
        for pos in part_of_speech:
            tx.run(
                """
                MATCH (w:Word {word: $word})
                MERGE (p:PartOfSpeech {name: $pos})
                MERGE (w)-[:HAS_PART_OF_SPEECH]->(p)
                """,
                word=word, pos=pos
            )

        # Create Definition Nodes and Relationships
        for definition in definitions:
            meaning = definition["meaning"]
            examples = definition["examples"]

            tx.run(
                """
                MATCH (w:Word {word: $word})
                MERGE (d:Definition {meaning: $meaning})
                SET d.examples = $examples
                MERGE (w)-[:HAS_DEFINITION]->(d)
                """,
                word=word, meaning=meaning, examples=examples
            )

if __name__ == "__main__":
    # Example data
    word_data = [
        {
            "word": "husky",
            "definitions": [
                {"meaning": "(of a voice) Hoarse and rough-sounding.", "examples": []},
                {"meaning": "Burly, stout.", "examples": []},
                {"meaning": "Abounding with husks; consisting of husks.", "examples": []}
            ],
            "part_of_speech": ["ADJECTIVE", "NOUN"],
            "sentiment_score": 0.006,
            "sentiment_magnitude": 0.059,
            "sentiment_type": "Neutral",
            "language": "en"
        },
        {
            "word": "hutan",
            "definitions": [
                {"meaning": "kawasan yg ditumbuhi pokok-pokok liar (terbiar), rimba",
                 "examples": ["mereka pergi ke ~ menahan perangkap", "pengawas ~ pegawai yg ditugaskan oleh kerajaan utk mengawasi hutan"]}
            ],
            "part_of_speech": ["KATA_NAMA"],
            "sentiment_score": 0.014,
            "sentiment_magnitude": 0.279,
            "sentiment_type": "Neutral",
            "language": "ms"
        }
    ]

    # Connect to Neo4j
    uri =  "neo4j+s://a9ec9647.databases.neo4j.io" # Change to your Neo4j instance URI
    user = "neo4j"                 # Default username
    password = "c4gsA4xCjixFJqaBwBtQeIQeKpkhotXQmK8O3Jy3e3o"          # Your Neo4j password

    store = Neo4jWordStore(uri, user, password)
    store.store_word_data(word_data)
    store.close()

/tmp/ipykernel_89221/3368978501.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_word_node, word_entry)
